In [1]:
import pandas as pd
import numpy as np

import pickle
import os
import random

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [2]:
seq_df = pd.read_table('data/family_classification_sequences.tab')
seq_df.head()

,Sequences
0,MAFSAEDVLKEYDRRRRMEALLLSLYYPNDRKLLDYKEWSPPRVQV...
1,MSIIGATRLQNDKSDTYSAGPCYAGGCSAFTPRGTCGKDWDLGEQT...
2,MQNPLPEVMSPEHDKRTTTPMSKEANKFIRELDKKPGDLAVVSDFV...
3,MDSLNEVCYEQIKGTFYKGLFGDFPLIVDKKTGCFNATKLCVLGGK...
4,MEAKNITIDNTTYNFFKFYNINQPLTNLKYLNSERLCFSNAVMGKI...


In [3]:
class CodoneTransformer:
    
    def __init__(self):
        self.len = ord('Z') - ord('A') + 1
        self.codones = {}
        
    def set_codones_dict(self, dict_copy):
        self.codones = dict_copy.copy()
    
    def to_int(self, codone):
        res = 0
        for i in range(3):
            res += (ord(codone[i]) - ord('A')) * (self.len ** i)
        self.codones[res] = codone
        return res
    
    def to_codone(self, intt):
        assert(intt in self.codones)
        return self.codones[intt]

codone_transformer = CodoneTransformer()

In [4]:
def make_codones(sseq):
    crop = len(sseq) % 3
    cropped_seq = sseq[:-crop] if crop > 0 else sseq

    return [codone_transformer.to_int(cropped_seq[i:i+3]) for i in range(0, len(cropped_seq), 3)]

def seq_to3(seq):
    splittings = [make_codones(seq[i:]) for i in range(3)]
    return splittings

def create_all_codones(df):
    codones = []

    for i in range(7000):
        row = df.iloc[i, :][0]
        codones.extend(seq_to3(row))
    return codones

In [5]:
def read_or_create(read_path, producer):
    if os.path.isfile(read_path):
        print('reading', read_path)
        with open(read_path, 'rb') as fp:
            return pickle.load(fp)
    result = producer()
    print('saving', read_path)
    with open(read_path, 'wb') as fp:
        pickle.dump(result, fp)
    return result

In [6]:
all_codones = read_or_create(read_path='data/all_codones.pickle',
                             producer= lambda: create_all_codones(seq_df))
codone_transformer.set_codones_dict(read_or_create(read_path='data/codones_to_int.pickle',
                                                  producer= lambda: codone_transformer.codones))

reading data/all_codones.pickle
reading data/codones_to_int.pickle


In [8]:
def generate_sample(index_words_list, context_window_size):
    """ Form training pairs according to the skip-gram model. """
    for index_words in index_words_list:
        for index, center in enumerate(index_words):
            context = random.randint(1, context_window_size)
            # get a random target before the center word
            for target in index_words[max(0, index - context): index]:
                yield center, target
            # get a random target after the center wrod
            for target in index_words[index + 1: index + context + 1]:
                yield center, target


def get_batch(iterator, batch_size):
    """ Group a numerical stream into batches and yield them as Numpy arrays. """
    while True:
        center_batch = np.zeros(batch_size, dtype=np.int32)
        target_batch = np.zeros([batch_size, 1], dtype=np.int32)
        for index in range(batch_size):
            center_batch[index], target_batch[index] = next(iterator)
        yield center_batch, target_batch


def flatten(x):
    return [item for sublist in x for item in sublist]


def cod_to_dict(cod, dictionary):
    return [dictionary[key] for key in cod]

def make_dictionary(all_codones):
    flat_codones = flatten(all_codones)
    unique_codones = set(flat_codones)
    dictionary = {cod: i for i, cod in enumerate(unique_codones)}
    return dictionary

def process_data(all_codones, dictionary, batch_size, skip_window):
    cod_dicts = [cod_to_dict(cod, dictionary) for cod in all_codones]
    single_gen = generate_sample(cod_dicts, context_window_size=skip_window)
    batch_gen = get_batch(single_gen, batch_size=batch_size)
    return batch_gen

In [9]:
dictionary = make_dictionary(all_codones)
count = [0] * len(dictionary)
new_codones = flatten(all_codones)
for codone in new_codones:
    count[dictionary[codone]] += 1

In [10]:
BATCH_SIZE = 128
SKIP_WINDOW = 12  # the context window
VOCAB_SIZE = len(dictionary)
EMBED_SIZE = 100  # dimension of the word embedding vectors
NUM_SAMPLED = 5  # Number of negative examples to sample.
LEARNING_RATE = .02
NUM_TRAIN_STEPS = 200000
SKIP_STEP = 1000

batch_gen = process_data(all_codones, dictionary, BATCH_SIZE, SKIP_WINDOW)

In [11]:
class SkipGramModel(nn.Module):
    
    def __init__(self, vocab_size, embed_size):
        super(SkipGramModel, self).__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.u = nn.Embedding(vocab_size, embed_size)
        self.v = nn.Embedding(vocab_size, embed_size)
        self.u.weight.data.uniform_(-0.5 / embed_size, 0.5 / embed_size)
        self.v.weight.data.uniform_(-0, 0)
                  
    def forward(self, u_pos, v_pos, v_neg):
        u = self.u(u_pos)
        v = self.v(v_pos)
        res = 0.0
        
        score = torch.mul(u, v).squeeze().sum(dim=1)
        res += F.logsigmoid(score).sum()
        
        v = self.v(v_neg)
        score = torch.bmm(v, u.unsqueeze(2)).squeeze()
        res += F.logsigmoid(-score).sum()
        return -res / BATCH_SIZE

In [12]:
def train_model(model):
    optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)
    total_loss = 0.0
    threshold = 13.5 * SKIP_STEP
    for step in range(NUM_TRAIN_STEPS):
        u, v = next(batch_gen)
        v_neg = np.random.choice(VOCAB_SIZE, size=(BATCH_SIZE, NUM_SAMPLED))
        u = torch.tensor(u, dtype=torch.long)
        v = torch.tensor(v, dtype=torch.long)
        v_neg = torch.tensor(v_neg, dtype=torch.long)
        optimizer.zero_grad()
        
        loss = model.forward(u, v, v_neg)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        if step % SKIP_STEP == 0 and step > 0:
            print('Average loss on step %d is %.3f' % (step, total_loss / SKIP_STEP))
            if total_loss <= threshold:
                break  # Because it's too long to wait and it probably won't get smaller
            total_loss = 0.0

    return model.u.weight.data.numpy()

In [13]:
model = SkipGramModel(VOCAB_SIZE, EMBED_SIZE)
final_embed_matrix = read_or_create(read_path='data/embeddings.pickle',
                                    producer=lambda: train_model(model))

reading data/embeddings.pickle


In [15]:
tsne = TSNE(n_components=2, random_state=42)
XX = tsne.fit_transform(final_embed_matrix)

In [17]:
tsne_df = pd.DataFrame(XX, columns=['x0', 'x1'])
unique_codones_ints = sorted(dictionary, key=dictionary.get)
unique_codones = []
for intt in list(unique_codones_ints):
    unique_codones.append(codone_transformer.to_codone(intt))    
tsne_df['codone'] = list(unique_codones)
tsne_df.head()

,x0,x1,codone
0,47.844692,51.136932,AAA
1,42.412998,-39.041664,CAA
2,66.300194,39.153183,DAA
3,59.061016,42.883232,EAA
4,-23.295715,42.688427,FAA


In [18]:
def plot_tsne_df(df):
    plt.figure(figsize=(15, 10))
    plt.title('unlabeled encoding', fontsize=20)
    plt.scatter(df.x0, df.x1, s=10)
    plt.show()

In [19]:
plot_tsne_df(tsne_df)

In [20]:
filename = 'data/acid_properties.csv'
props = pd.read_csv(filename)

In [21]:
def acid_dict(some_c, props):
    prop_by_letter = [props[props.acid == let].iloc[:, 1:] for let in some_c]   
    df_concat = pd.concat(prop_by_letter)
    res = df_concat.mean()
    dres = dict(res)
    dres['acid'] = some_c
    return dres

In [22]:
save_path = 'data/all_acid_dicts.pickle'
producer = lambda: [acid_dict(some_c, props) for some_c in tsne_df.codone]
all_acid_dicts = read_or_create(save_path, producer)

saving data/all_acid_dicts.pickle


In [23]:
all_acid_df = pd.DataFrame(all_acid_dicts)
all_acid_df.head()

,acid,hydrophobicity,mass,number_of_atoms,volume
0,AAA,1.800000,71.077900,13.000000,88.600000
1,CAA,2.033333,81.766233,13.333333,95.233333
2,DAA,0.033333,85.747733,14.000000,96.100000
3,EAA,0.033333,90.423267,15.000000,105.200000
4,FAA,2.133333,96.443233,16.333333,122.366667


In [24]:
final_df = all_acid_df.join(tsne_df.set_index('codone'), on='acid')
final_df.head()

,acid,hydrophobicity,mass,number_of_atoms,volume,x0,x1
0,AAA,1.800000,71.077900,13.000000,88.600000,47.844692,51.136932
1,CAA,2.033333,81.766233,13.333333,95.233333,42.412998,-39.041664
2,DAA,0.033333,85.747733,14.000000,96.100000,66.300194,39.153183
3,EAA,0.033333,90.423267,15.000000,105.200000,59.061016,42.883232
4,FAA,2.133333,96.443233,16.333333,122.366667,-23.295715,42.688427


In [25]:
def plot_embedding_properties(final_df):
    plt.figure(figsize=(25, 20))
    for i, p in enumerate(['hydrophobicity', 'mass', 'number_of_atoms', 'volume']):
        plt.subplot(2,2,i+1)
        plt.title(p, fontsize=25)
        plt.scatter(final_df.x0, final_df.x1, c=final_df[p], s=10)
    plt.show()

In [26]:
plot_embedding_properties(final_df)

In [27]:
filename = 'data/nice_embed_tsne.csv'
gensim_tsne_df = pd.read_csv(filename, index_col=0)
gensim_tsne_df.columns = ['x0', 'x1', 'codone']

In [28]:
plot_tsne_df(gensim_tsne_df)

In [29]:
final_df_nice = all_acid_df.join(gensim_tsne_df.set_index('codone'), on='acid')

In [31]:
plot_embedding_properties(final_df_nice)